In [ ]:
# 时间匆忙，本来想使用文本分类完成本次作业的
# -*- coding: utf-8 -*-
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

import time

#定义超参数
batch_size = 32
num_epoches = 20
learning_rate = 1e-3


#1.引入模块，读取数据 """
#下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



#2.构建计算图（构建网络模型） """
#建立网络
class Logistic_Regression(nn.Module):
    """逻辑回归模型定义"""

    def __init__(self, in_dim, n_class):
        super(Logistic_Regression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        #前向传播
        output = self.logistic(x)
        return output

#模型初始化
model = Logistic_Regression(28 * 28, 10)   #图片大小是28*28



""" 3.损失函数与优化器 """
#定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)



""" 4.开始训练模型 """
#训练模型
for epoch in range(num_epoches):
    print ('#' * 45)
    print ('Epoch{}'.format(epoch + 1))
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data, in enumerate(test_loader, 1):
        img, lable = data
        img = img.view(img.size(0), -1)

        img = Variable(img)
        lable = Variable(lable)

        #前向传播
        out = model(img)
        loss = criterion(out, lable)
        running_loss += loss.item() * lable.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == lable).sum()
        running_acc += num_correct.item()

        #后向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print ('[{} / {}] Loss : {:.6f}, Acc : {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print ('Finish {} epoch, Loss : {:.6f}, Acc : {:.6f}'.format(
        epoch + 1, num_epoches, running_loss / (len(train_dataset)),
        running_acc / (len(train_dataset))))



    """ 5.对训练的模型预测结果进行评估 """
    #模型评估
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, lable = data
        img = img.view(img.size(0), -1)

        with torch.no_grad():
            img = Variable(img)
            lable = Variable(lable)

        out = model(img)
        loss = criterion(out, lable)
        eval_loss += loss.item() * lable.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == lable).sum()
        eval_acc += num_correct.item()
    print ('Test Loss : {:.6f}, ACC : {:.6f}'.format(
        eval_loss / (len(test_dataset)), eval_acc / (len(test_dataset))))
    print ('Time : {:1f} s'.format(time.time() - since))